In [1]:
%load_ext line_profiler

In [2]:
import rankBN
import numpy as np

In [3]:
%lprun -f rankBN.Ranker._check_common_ground rankBN.do_main("../../test/benzene.xyz", "benzene.mol2")

/home/guido/miniconda3/envs/analysis/lib/python3.6/site-packages/MDAnalysis/topology/guessers.py:72: UserWarning: Failed to guess the mass for the following atom types: C.ar
  warnings.warn("Failed to guess the mass for the following atom types: {}".format(atom_type))


Working on stoichiometry: [5 5 5 7 7 7]


  0%|          | 0/90 [00:00<?, ?it/s]

Found 20 permutations.
Found 3 molecules in 3 connected components.
Found: (5, 7, 5, 7, 5, 7)
Found: (5, 7, 5, 7, 7, 5)
Found: (5, 5, 7, 7, 7, 5)
Working on stoichiometry: [5 5 7 7 6 6]
Found 90 permutations.


 37%|███▋      | 11/30 [00:00<00:00, 109.33it/s]

Found 11 molecules in 6 connected components.
Found: (6, 7, 5, 7, 5, 6)
Found: (6, 7, 5, 6, 7, 5)
Found: (6, 7, 5, 6, 5, 7)
Found: (7, 6, 5, 7, 5, 6)
Found: (7, 5, 7, 6, 5, 6)
Found: (6, 6, 5, 7, 7, 5)
Found: (7, 6, 6, 7, 5, 5)
Found: (6, 5, 5, 7, 6, 7)
Found: (7, 7, 5, 6, 5, 6)
Found: (5, 5, 7, 7, 6, 6)
Found: (7, 7, 6, 5, 5, 6)
Working on stoichiometry: [5 7 6 6 6 6]
Found 30 permutations.


100%|██████████| 1/1 [00:00<00:00, 337.84it/s]

Found 3 molecules in 2 connected components.
Found: (6, 6, 7, 5, 6, 6)
Found: (6, 5, 6, 6, 6, 7)
Found: (6, 6, 5, 6, 6, 7)
Working on stoichiometry: [6 6 6 6 6 6]
Found 1 permutations.
Found 1 molecules in 1 connected components.
Found: (6, 6, 6, 6, 6, 6)


Timer unit: 1e-06 s

Total time: 0.847659 s
File: /mnt/c/Users/guido/workcopies/apdft/prototyping/networks/rankBN.py
Function: _check_common_ground at line 220

Line #      Hits         Time  Per Hit   % Time  Line Contents
   220                                           	def _check_common_ground(self, target, opposite, reference, common_ground):
   221      9401      89436.0      9.5     10.6  		deltaZ = opposite - target
   222                                           
   223                                           		# matching deltaZ
   224      9401     150528.0     16.0     17.8  		changes = np.bincount([_ + 2 for _ in deltaZ], minlength=5)
   225                                           
   226                                           		# ensure matching counts
   227      9401     114217.0     12.1     13.5  		if max(changes - changes[::-1]) != 0:
   228      4020      12722.0      3.2      1.5  			return False
   229                                           
   230      

In [4]:
deltaZ = (0,0,1,-1,2,-2, 0, 2)
%timeit np.unique(deltaZ, return_counts=True)

23.3 µs ± 722 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [15]:
q = np.bincount(np.array(deltaZ)+2, minlength=5)
if max(q - q[::-1]) != 0:
    print ("dang")

In [23]:
%timeit np.bincount(np.array(deltaZ)+2, minlength=5)

4.2 µs ± 524 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [22]:
%timeit np.bincount([_+2 for _ in deltaZ], minlength=5)

2.88 µs ± 124 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [5]:
q = np.bincount([_+2 for _ in deltaZ], minlength=5)

In [8]:
%timeit max(q - q[::-1])

2.98 µs ± 144 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [9]:
import igraph as ig

In [34]:
g = ig.Graph()
g.add_vertex(0)
g.add_vertex(1)
g.add_vertex(2)
g.are_connected(0, 1)

False

In [40]:
g.add_edge(0, 1)
g.add_edge(2, 1)

In [15]:
g.are_connected(0, 2)

False

In [16]:
help(g.shortest_paths)

Help on built-in function shortest_paths:

shortest_paths(...) method of igraph.Graph instance
    shortest_paths(source=None, target=None, weights=None, mode=OUT)
    
    Calculates shortest path lengths for given vertices in a graph.
    
    The algorithm used for the calculations is selected automatically:
    a simple BFS is used for unweighted graphs, Dijkstra's algorithm is
    used when all the weights are positive. Otherwise, the Bellman-Ford
    algorithm is used if the number of requested source vertices is larger
    than 100 and Johnson's algorithm is used otherwise.
    
    @param source: a list containing the source vertex IDs which should be
      included in the result. If C{None}, all vertices will be considered.
    @param target: a list containing the target vertex IDs which should be
      included in the result. If C{None}, all vertices will be considered.
    @param weights: a list containing the edge weights. It can also be
      an attribute name (edge weight

In [36]:
import math
q = g.shortest_paths(0, 2)[0][0]

In [37]:
%timeit math.isinf(q)

205 ns ± 42.5 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


In [41]:
for c in g.components():
    print (c)

[0, 1, 2]


In [42]:
help(np.outer)

Help on function outer in module numpy:

outer(a, b, out=None)
    Compute the outer product of two vectors.
    
    Given two vectors, ``a = [a0, a1, ..., aM]`` and
    ``b = [b0, b1, ..., bN]``,
    the outer product [1]_ is::
    
      [[a0*b0  a0*b1 ... a0*bN ]
       [a1*b0    .
       [ ...          .
       [aM*b0            aM*bN ]]
    
    Parameters
    ----------
    a : (M,) array_like
        First input vector.  Input is flattened if
        not already 1-dimensional.
    b : (N,) array_like
        Second input vector.  Input is flattened if
        not already 1-dimensional.
    out : (M, N) ndarray, optional
        A location where the result is stored
    
        .. versionadded:: 1.9.0
    
    Returns
    -------
    out : (M, N) ndarray
        ``out[i, j] = a[i] * b[j]``
    
    See also
    --------
    inner
    einsum : ``einsum('i,j->ij', a.ravel(), b.ravel())`` is the equivalent.
    ufunc.outer : A generalization to N dimensions and other operations.
 

In [59]:
tpl = '''calc=ccsd(t)
mem=5GB
core=frozen
ccprog=ccsd
basis=cc-pvdz
unit=angs
geom=xyz
18

%s          0.00000        0.70760        0.00010
%s          0.00000       -0.70760        0.00000
%s          1.22500        1.39440        0.00000
%s          1.22500       -1.39440        0.00010
%s         -1.22500        1.39430        0.00000
%s         -1.22500       -1.39430        0.00000
%s          2.43270        0.69590       -0.00010
%s          2.43270       -0.69580        0.00000
%s         -2.43270        0.69580        0.00000
%s         -2.43270       -0.69580       -0.00010
H          1.24890        2.48210       -0.00010
H          1.24890       -2.48220        0.00010
H         -1.24900        2.48210        0.00010
H         -1.24890       -2.48220       -0.00010
H          3.37320        1.23910       -0.00010
H          3.37330       -1.23900       -0.00010
H         -3.37320        1.23900        0.00000
H         -3.37330       -1.23900       -0.00010'''

In [45]:
lines = open('n.log').readlines()

In [58]:
args = [[{'5': 'B', '6': 'C', '7': 'N'}[__] for __ in _.strip()[8:-1].replace(' ','').split(',')] for _ in lines if 'Found:' in _]

In [71]:
import os
print (''.join(args[0]))
print ( tpl % (*args[0],))

BBBBBNNNNN
calc=ccsd(t)
mem=5GB
core=frozen
ccprog=ccsd
basis=cc-pvdz
unit=angs
geom=xyz
18

B          0.00000        0.70760        0.00010
B          0.00000       -0.70760        0.00000
B          1.22500        1.39440        0.00000
B          1.22500       -1.39440        0.00010
B         -1.22500        1.39430        0.00000
N         -1.22500       -1.39430        0.00000
N          2.43270        0.69590       -0.00010
N          2.43270       -0.69580        0.00000
N         -2.43270        0.69580        0.00000
N         -2.43270       -0.69580       -0.00010
H          1.24890        2.48210       -0.00010
H          1.24890       -2.48220        0.00010
H         -1.24900        2.48210        0.00010
H         -1.24890       -2.48220       -0.00010
H          3.37320        1.23910       -0.00010
H          3.37330       -1.23900       -0.00010
H         -3.37320        1.23900        0.00000
H         -3.37330       -1.23900       -0.00010


In [72]:
for arg in args:
    label = ''.join(arg)
    os.mkdir('naphtalene/validation/%s' % label)

In [74]:
for arg in args:
    label = ''.join(arg)
    with open('naphtalene/validation/%s/MINP' % label, 'w') as fh:
        fh.write(tpl % (*arg,))